### MLP with Tensorboard 2

see:  https://github.com/nlintz/TensorFlow-Tutorials/blob/master/9_tensorboard.py

testing optimization loss function and gradient clipping

https://github.com/tensorflow/tensorflow/blob/c936c06a20b867eaa868e9f2c4fb53856391b18f/tensorflow/contrib/layers/python/layers/optimizers.py#L114

https://github.com/kapild/deeplearning/blob/8adb8452a8b16e15bdcec23153a78a3b1fa02f76/tensorflow/contrib/layers/python/layers/optimizers_test.py

In [1]:
# A Multilayer Perceptron implementation example using TensorFlow library.
# This example is using the MNIST database of handwritten digits
# (http://yann.lecun.com/exdb/mnist/)=


# Author: Aymeric Damien
# Project: https://github.com/aymericdamien/TensorFlow-Examples/

In [2]:
import tensorflow as tf
import numpy as np
import input_data

In [3]:
# Parameters
#learning_rate = 0.001
training_epochs = 100
batch_size = 100
display_step = 1
total_batch = 55000/100

In [4]:
# Network Parameters
n_hidden_1 = 1000 # 1st layer num features
n_hidden_2 = 500 # 2nd layer num features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

std_0 = 1.0/np.sqrt(n_input)
std_h1 = 1.0/np.sqrt(n_hidden_1)
std_h2 = 1.0/np.sqrt(n_hidden_2)


In [5]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [6]:
def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2'])) 
    return tf.matmul(layer_2, weights['out']) + biases['out']

In [7]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=std_0)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=std_h1)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=std_h2))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1], stddev=std_h1)),
    'b2': tf.Variable(tf.random_normal([n_hidden_2], stddev=std_h2)),
    'out': tf.Variable(tf.random_normal([n_classes], stddev=0.3))
}

In [8]:
# Construct model
pred = multilayer_perceptron(x, weights, biases )

In [9]:
cost =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) 
gs = tf.get_variable("global_step",[],trainable=False,initializer=tf.constant_initializer(0))
lr = tf.constant(0.001) 

In [10]:
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import clip_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import logging_ops
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.ops import variables as vars_
from tensorflow.python.training import training as train

In [11]:
# we can check to see if we built the model sanely

In [12]:
variables = vars_.trainable_variables()
for v in variables:
    print v.name

Variable:0
Variable_1:0
Variable_2:0
Variable_3:0
Variable_4:0
Variable_5:0


In [13]:
train_op = tf.contrib.layers.optimize_loss(cost, global_step=gs, learning_rate=lr,optimizer="SGD")

In [14]:
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)) # Count correct predictions
    # Calculate accuracy
    acc_op = tf.reduce_mean(tf.cast(correct_prediction, "float"))  # Cast boolean to float to average
    # Add scalar summary for accuracy
    tf.scalar_summary("accuracy", acc_op)

In [15]:
mnist = input_data.read_data_sets(".", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

/Users/charlesmartin14/anaconda2/lib/python2.7/gzip.py:275: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  chunk = self.extrabuf[offset: offset + size]
input_data.py:35: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


In [16]:
print trX.shape, trY.shape
print teX.shape, teY.shape

(55000, 784) (55000, 10)
(10000, 784) (10000, 10)


In [17]:
55000./125.

440.0

In [18]:
!rm -rf ./logs/nn_logs

In [ ]:
with tf.Session() as sess:
    # create a log writer. run 'tensorboard --logdir=./logs/nn_logs'
    writer = tf.train.SummaryWriter("./logs/nn_logs", sess.graph) # for 0.8
    merged = tf.merge_all_summaries()

    # you need to initialize all variables
    tf.initialize_all_variables().run()

    for i in range(5000):
        for start, end in zip(range(0, len(trX), 125), range(125, len(trX), 125)):
            #print start, end
            sess.run(train_op, feed_dict={x: trX[start:end], y: trY[start:end]})
        summary , acc = sess.run([merged, acc_op], feed_dict={x: teX, y: teY})
        writer.add_summary(summary, i)  # Write summary
        print(i, acc)                   # Report the accuracy
        
    writer.flush()

(0, 0.41209999)
(1, 0.6153)
(2, 0.69010001)
(3, 0.72729999)
(4, 0.75830001)
(5, 0.78250003)
(6, 0.8021)
(7, 0.81720001)
(8, 0.83020002)
(9, 0.84069997)
(10, 0.85049999)
(11, 0.85610002)
(12, 0.8628)
(13, 0.86729997)
(14, 0.87260002)
(15, 0.87669998)
(16, 0.88)
(17, 0.88330001)
(18, 0.88590002)
(19, 0.88849998)
(20, 0.88999999)
(21, 0.89209998)
(22, 0.89389998)
(23, 0.89529997)
(24, 0.8962)
(25, 0.89679998)
(26, 0.89749998)
(27, 0.89810002)
(28, 0.89840001)
(29, 0.90030003)
(30, 0.90189999)
(31, 0.90390003)
(32, 0.9048)
(33, 0.90549999)
(34, 0.90679997)
(35, 0.9073)
(36, 0.90810001)